# Desafio

Executar o fine-tuning de um foudation model (Llama, Bert, Minstrel, etc), utilizando o dataset "TheAmazonTtiles-1.3MM". <br>
O modelo treinado deverá:
- Receber perguntas com um contexto obtido por meio de uma integração RAG (Retrieve-and-Generate), utilizando documentos relacionados aos produtos da Amazon.
- A partir do prompt formado pela pergunta do usuário e dos dados retornados do RAG, o modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do RAG, incluindo as fontes.

# Passos para a aplicação de fine-tuning no Modelo

O The AmazonTitles-1.3MM consiste em consultas textuais reais de usuários e títulos associados de produtos relevantes encontrados na Amazon, medidos por ações implícitas ou explícitas dos usuários.
1.   Preparação do Dataset <br>
    - Download do dataset AmazonTitles-1.3MM.
    - Prepare os dados para o fine-tuning, garantindo que estejam organizados de maneira adequada para o treinamento do modelo.
    - Limpe e pré-processe os dados conforme necessário para o modelo escolhido.
2.  Execução do Fine-Tuning <br>
    - Execute o fine-tuning do foundation model selecionado utilizando o dataset preparado.
    - Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.
3.  Configuração da Integração RAG
    - Configure uma integração RAG (Retrieve-and-Generate) para fornecer contexto ao modelo a partir dos documentos relacionados aos produtos da Amazon.
    - Certifique-se de que a integração esteja funcionando corretamente para recuperar e fornecer dados contextuais ao modelo.
4.  Geração de Respostas
    - Configure o modelo treinado para receber perguntas dos usuários.
    - Quando uma pergunta for recebida, utilize a integração RAG para recuperar informações relevantes do dataset AmazonTitles-1.3MM.
    - Combine a pergunta do usuário e os dados retornados do RAG para formar um prompt completo.
    - O modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do RAG, incluindo as fontes fornecidas.

# Preparação do Dataset
- Download do dataset AmazonTitles-1.3MM.
- Prepare os dados para o fine-tuning, garantindo que estejam organizados de maneira adequada para o treinamento do modelo.
- Limpe e pré-processe os dados conforme necessário para o modelo escolhido.

In [28]:
FOLDER_PATH = '/content/drive/MyDrive/TheAmazonTitles'
FINE_TUNING_PATH = f'{FOLDER_PATH}/fine-tuning'
URLDATA_GOOGLEDRIVE = f'https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK'

In [2]:
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


Dado que os dados disponibilizados para construção do modelo encontram-se em um link do Google Drive, se faz necessário obter alguma estratégia para manuseio.
No modelo, utilizando a lib gdown, é feito o download a partir do link e os arquivos extraídos para uma pasta em MyDrive.
Se faz necessária essa estratégia pois garante que, ao encerrar a sessão, os dados não são perdidos, gerando a necessidade de realizar o download e extração novamente.

In [7]:
import gdown
import zipfile
import os

if not os.path.exists(FOLDER_PATH):
    os.makedirs(FOLDER_PATH)

zip_file = f'{FOLDER_PATH}/LF-Amazon-1.3M.zip'

gdown.download(URLDATA_GOOGLEDRIVE, zip_file, quiet=False)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(FOLDER_PATH)

print("Files successfully extracted and saved to the Google Drive directory!")

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=50397530-b385-46d7-8e0d-0fedd5e4902e
To: /content/drive/MyDrive/TheAmazonTitles/LF-Amazon-1.3M.zip
100%|██████████| 890M/890M [00:13<00:00, 67.5MB/s]


Arquivos extraídos e salvos no diretório no Google Drive com sucesso!


Estrutura do Dowload de AmazonTitles-1.3MM é a seguinte:

- pasta.zip e a mesma abriga os seguintes arquivos:
    - filter_labels_train.txt (padrão do conteúdo do arquivo: número número)
    - filter_labels_test.txt (padrão do conteúdo do arquivo: número número)
    - tst.json.gz (padrão do conteúdo do arquivo: objeto json inline)
    - trn.json.gz (padrão do conteúdo do arquivo: objeto json inline)
    - lbl.json.gz (padrão do conteúdo do arquivo: objeto json inline)

Todos os arquivos acima passam das 100 mil linhas cada um. <br>
Para lidar com arquivos grandes como os .json.gz, carregar todo o conteúdo na memória pode ser inviável por falta de recursos computacionais. <br>
A abordagem adotada foi aplicar o algoritmo de Reservoir Sampling: permite selecionar uma amostra aleatória de tamanho fixo de um fluxo de dados de tamanho desconhecido sem carregar todo o conteúdo do arquivo em memória.

In [11]:
import gzip
import random
import json

def reservoir_sampling_gz_json_with_indices(file_path, sample_size):
    sample = []
    indices = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line)
                if i < sample_size:
                    sample.append(obj)
                    indices.append(i)
                else:
                    j = random.randint(0, i)
                    if j < sample_size:
                        sample[j] = obj
                        indices[j] = i
            except json.JSONDecodeError as e:
                print(f"Erro ao decodificar JSON na linha {i}: {e}")
    return sample, indices


In [46]:
# Tamanho das amostras desejadas
sample_size_train = 4000
sample_size_test = 1000
sample_size_label = 4000

# Realiza a amostragem dos arquivos gzip de treinamento e teste
train_sample, train_indices = reservoir_sampling_gz_json_with_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/trn.json.gz', sample_size_train)
test_sample, test_indices = reservoir_sampling_gz_json_with_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/tst.json.gz', sample_size_test)

print(len(train_sample))
print(len(test_sample))

4000
1000


Após a extração dos dados de treino e teste, é o momento de extrair o conteúdo dos arquivos de filter_labels para indexar com os dados extraídos anteriormente. E assim criar um mapeamento dos dados com base nos filtros.

In [52]:
# Carregar os labels correspondentes aos índices
def load_labels_by_indices(file_path, indices):
    labels = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
    for idx in indices:
        if idx < len(lines):
            labels.append(lines[idx].strip().split())
        else:
            labels.append([])
    return labels

filter_labels_train_sample = load_labels_by_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/filter_labels_train.txt', train_indices)
filter_labels_test_sample = load_labels_by_indices(f'{FOLDER_PATH}/LF-Amazon-1.3M/filter_labels_test.txt', test_indices)

print(len(filter_labels_train_sample))
print(filter_labels_train_sample[:10])

4000
[[], ['942479', '814763'], ['630940', '249347'], [], [], ['10286', '10679'], [], ['1008750', '507059'], [], []]


## Realizando a limpeza dos dados

É de extrema importância a etapa de limpeza, pois é a partir dela que geramos inputs sem ruídos para o treinamento do modelo.
Abaixo, foi implementada limpeza de HTML Entities nos campos de texto, além de outros tratamentos necessários como remoção de novas linhas e múltiplos espaços.

In [55]:
import re
import html

def unescape_html_entities(text):
    text = html.unescape(text)
    def replace_hex_entity(match):
        code_point = int(match.group(1), 16)
        return chr(code_point)

    text = re.sub(r'&#x([0-9A-Fa-f]+);', replace_hex_entity, text)
    return text

def clean_text(text):
    text = unescape_html_entities(text)
    text = text.replace('’', "'")
    # Remove caracteres especiais
    text = re.sub(r"[^A-Za-z0-9' ]+", '', text)
    # Remove novas linhas e espaços extras
    text = text.replace('\n', ' ').replace('\r', ' ').strip()
    # Remove múltiplos espaços
    text = re.sub(' +', ' ', text)
    return text


In [62]:
from typing import List, Dict

# Processa uma lista de amostras, limpando textos e associando rótulos. Remove itens que não possuem 'title' ou 'uid'.
def process_samples(sample_data: List[Dict], filter_labels_sample: List) -> List[Dict]:
    processed_samples = []
    total_items = len(sample_data)
    total_labels = len(filter_labels_sample)

    for idx, item in enumerate(sample_data):
        title = item.get('title', '').strip()
        uid = item.get('uid', '').strip()

        if not title or not uid:
            continue  # Ignora itens com 'title' ou 'uid' vazios

        # Limpeza dos campos de texto
        item['title'] = clean_text(title)
        item['content'] = clean_text(item.get('content', ''))

        # Associação de rótulos
        if idx < len(filter_labels_sample):
            item['labels'] = filter_labels_sample[idx]
        else:
            item['labels'] = []

        processed_samples.append(item)

    return processed_samples

In [64]:
clean_train_sample = process_samples(train_sample, filter_labels_train_sample)
clean_test_sample = process_samples(test_sample, filter_labels_test_sample)

print(clean_train_sample[0])
print(clean_test_sample[0])

{'uid': 'B009HV1TI0', 'title': 'FIELD ASSEMBLY THREADED TENSIONER For 18 Cable Railing', 'content': 'This wire rope cable assembly is a great choice for the installerhomeowner to cut cable and swage fittings at the job site You can order the assemblies before your posts are in and the construction is finished', 'target_ind': [846871, 1037714], 'target_rel': [1.0, 1.0], 'labels': []}
{'uid': 'B000086778', 'title': 'Braza Happy Straps', 'content': '', 'target_ind': [114755, 114756, 583947, 592065, 751696, 751699, 751779, 751782, 751787, 751789, 751790, 751791, 751792, 751796, 751798], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'labels': ['16462', '265932']}


# Execução do Fine-Tuning
- Execute o fine-tuning do foundation model selecionado utilizando o dataset preparado.
- Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.

In [21]:
!pip install datasets
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


Para esse modelo foi escolhido o distilgpt2 por ser uma versão menor e mais leve do GPT-2, projetada para ser mais eficiente em termos de recursos. Como o modelo é para fins acadêmicos, é o suficiente para aplicar as lógicas necessárias.
E para treinar esse modelo, são necessárias algumas etapas prévias:

1. Preparação dos Dados de Treinamento, processando amostras de dados limpos para criar pares de "prompt" e "completion".
2. Criação de um Dataset, convertendo essas amostras em um formato adequado para treinamento utilizando a biblioteca datasets.

In [22]:
from datasets import Dataset

train_data = clean_train_sample
test_data = clean_test_sample

# Prepara os dados para o dataset
training_samples = []
for item in train_data:
    prompt = f"Title: {item['title']}\n"
    completion = item['content']
    training_samples.append({'prompt': prompt, 'completion': completion})

# Cria Dataset
train_dataset = Dataset.from_list(training_samples)

In [23]:
train_dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 3772
})


3. Configuração e Carregamento do Modelo e Tokenizer, através do modelo pré-treinado distilgpt2 da biblioteca transformers. Ajusta o tokenizer e o modelo para garantir que estejam alinhados em termos de tokens.
4. Tokenização dos Dados, que transforma os textos de entrada em tokens numéricos que o modelo pode processar. Em linhas gerais, prepara os dados para serem alimentados no modelo durante o treinamento.

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'distilgpt2'

# Carrega o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
'''
Faz a atribuição do pad_token.
O GPT-2 originalmente não possue um token de padding,
e essa substituição ajuda na tokenização de sequências de comprimento fixo.
'''
tokenizer.pad_token = tokenizer.eos_token

# Carrega o modelo escolhido
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))  # Redimensiona o Token Embeddings

def tokenize_function(examples):
    # Concatena 'prompt' e 'completion' para cada item
    inputs = [i + t for i, t in zip(examples['prompt'], examples['completion'])]
    # Tokeniza os inputs
    tokenized_inputs = tokenizer(
        inputs,
        truncation=True, # Garante que sequências mais longas que max_length sejam truncadas.
        padding='max_length', # Adiciona padding para que todas as sequências tenham o mesmo comprimento
        max_length=128, # Define o comprimento máximo das sequências tokenizadas.
    )
    return tokenized_inputs


tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['prompt', 'completion']
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/3772 [00:00<?, ? examples/s]

In [25]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3772
})

In [26]:
from transformers import DataCollatorForLanguageModeling

'''
Configura um objeto responsável por preparar os dados em lotes adequados para o
treinamento do modelo de linguagem.
'''
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # Masked Language Modeling = False / Causal Language Modeling = True.
)


In [ ]:
from transformers import TrainingArguments, Trainer

# Configuração dos argumentos para Treinamento
training_args = TrainingArguments(
    output_dir=f'{FINE_TUNING_PATH}/training_args',
    num_train_epochs=3,  # Número de vezes que o conjunto de dados completo será passado pelo modelo durante o treinamento.
    per_device_train_batch_size=2, # Tamanho do lote por dispositivo (GPU/CPU) durante o treinamento.
    save_steps=10_000, # Intervalo de passos (steps) entre cada salvamento do modelo. Permite salvar checkpoints periódicos
    save_total_limit=2, # Número máximo de checkpoints a serem mantidos. Os mais antigos são removidos quando o limite é excedido.
    logging_steps=250, # Intervalo de passos entre cada registro (log) de métricas de treinamento.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Inicializando treino dos dados com o modelo distilgpt2
trainer.train()

Step,Training Loss
250,4.607400
500,4.534300
750,4.487400
1000,4.468800
1250,4.417500
1500,4.455400
1750,4.359600
2000,4.341300
2250,3.921600
2500,3.935100


A tendência geral de diminuição da perda indica que o modelo está aprendendo e melhorando seu desempenho ao longo do treinamento.
A estabilização da perda sugere que o modelo está alcançando um ponto de convergência. Se a perda não diminuir mais, pode ser necessário:
1. Aumentar o Número de Épocas: Talvez o modelo precise de mais treinamento para melhorar ainda mais.
2. Ajustar a Taxa de Aprendizado: Uma taxa de aprendizado menor pode ajudar a refinar o treinamento nos estágios finais.
3. Utilizar Técnicas de Regularização: Para evitar overfitting e melhorar a generalização.

In [29]:
# Salvando componentes do Treino após a execução para não retreinar a cada queda de sessão
trainer.save_model(f'{FINE_TUNING_PATH}/trainer-result')
model.save_pretrained(f'{FINE_TUNING_PATH}/model-tokenizer')
tokenizer.save_pretrained(f'{FINE_TUNING_PATH}/model-tokenizer')

('/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/vocab.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/merges.txt',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/TheAmazonTitles/fine-tuning/model-tokenizer/tokenizer.json')

# Configuração da Integração RAG
- Configure uma integração RAG (Retrieve-and-Generate) para fornecer contexto ao modelo a partir dos documentos relacionados aos produtos da Amazon.
- Certifique-se de que a integração esteja funcionando corretamente para recuperar e fornecer dados contextuais ao modelo.


In [35]:
!pip install langchain
!pip install -U langchain-community
!pip install sentence_transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 20.9 MB/s eta 0:00:00


## Preparação dos Documentos para o Armazenamento Vetorial

Organizar os dados em objetos Document facilita o gerenciamento e a manipulação dos documentos, especialmente quando se trata de adicionar metadados que podem ser úteis para buscas e filtragens futuras.
Além de que a estruturação dos documentos de forma consistente permite que os embeddings sejam gerados de maneira uniforme, garantindo que todas as informações relevantes (como título e UID) estejam disponíveis para uso posterior.

In [65]:
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

# Preparação dos documentos para o uso do Vector Store
documents = []
for item in train_sample:
    content = item['content']
    documents.append(Document(page_content=content, metadata={'uid': item['uid'], 'title': item['title']}))

documents[0]

Document(metadata={'uid': 'B009HV1TI0', 'title': 'FIELD ASSEMBLY THREADED TENSIONER For 18 Cable Railing'}, page_content='This wire rope cable assembly is a great choice for the installerhomeowner to cut cable and swage fittings at the job site You can order the assemblies before your posts are in and the construction is finished')

## Criação de Embeddings e Armazenamento Vetorial
Embeddings transformam texto em vetores numéricos que capturam semântica e contexto, permitindo que algoritmos de busca e recuperação encontrem similaridades entre diferentes textos de forma eficiente. E a lib FAISS permite indexar e buscar rapidamente vetores de alta dimensão, tornando-o ideal para aplicações que exigem buscas por similaridade em grandes conjuntos de dados, como sistemas de recomendação, busca semântica e recuperação de informações.

`all-MiniLM-L6-v2` é conhecido por ser eficiente e produzir embeddings de alta qualidade para tarefas de processamento de linguagem natural.

In [70]:
# Cria os embeddings e o vector store
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever()
retriever

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b3b3b686c80>, search_kwargs={})

Verificando se o retriver está hábil a consultar Documentos contextualizados com o prompt fornecido.

In [69]:
question = "Something about Super Hero"
docs = retriever.get_relevant_documents(question)
docs

[Document(metadata={'uid': '1401202691', 'title': 'Batman Illustrated Volume 2 Batman DC Comics Hardcover'}, page_content="ne of Adams' early triumphs involved revitalizing one of comics' greatest icons In the waning days of the TV seriesBatman which depicted the character as a genial battler against silly costumed villains Adams and some sympathetic writers returned to the crime fighter's roots as a dark creature of the night driven to avenge his parents' double murder Besides conjuring a noirish atmosphere intensified here by good recoloring Adams goosed up the thrills with exciting kinetic fight choreography The 1970s stories reprinted in this attractive pricey hardcover range from moody ghost stories set in haunted houses to more traditional superhero scenarios such as the trilogy in which Batman fights the ManBat a scientist transformed into a giant flying fiend More than three decades after these comics first appeared Adams' conception of Batman remains the template for today's r

## Reutilização de Modelos Fine-Tuned
Utiliza o modelo e tokenizer treinados previamente com o model `distilgpt2`. Permitindo o aproveitamento do conhecimento adquirido durante o fine-tuning, garantindo que o modelo esteja adaptado às necessidades específicas.

In [67]:
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(f'{FINE_TUNING_PATH}/model-tokenizer')
model = AutoModelForCausalLM.from_pretrained(f'{FINE_TUNING_PATH}/model-tokenizer')

nlp = pipeline('text-generation', model=model, tokenizer=tokenizer)
nlp

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


## Personalizando as Respostas com o RAG

E abaixo, os métodos responsáveis pela captação dos documentos e geração das respostas das questões inputadas pelo usuário.

Com esses métodos é possível:
1. Fornecer ao modelo um contexto relevante e conciso, aumentando a probabilidade de gerar respostas precisas e informativas.
2. Truncar o contexto garante que o prompt completo (incluindo a pergunta e o contexto) não exceda o limite.
3. Retornar as fontes permite que você saiba de onde as informações foram extraídas, o que é útil para verificação e transparência.

In [76]:
'''
Recupera e processa o contexto relevante para uma pergunta
1. Utiliza o retriever (configurado anteriormente com FAISS e embeddings) para buscar documentos que são semanticamente relevantes para a pergunta fornecida.
Construção do Contexto:
2. Combina o conteúdo (page_content) dos documentos recuperados em uma única string, separada por quebras de linha.
Truncamento do Contexto:
3. Verifica se o número de tokens no contexto excede max_context_tokens (500 tokens neste caso).
Se exceder, trunca o contexto para manter o comprimento dentro do limite, convertendo os tokens truncados de volta para uma string.
Coleta de Metadados (Fontes):
4. Extrai os metadados (metadata) dos documentos recuperados, que podem incluir informações como 'uid' e 'title'.
'''
def get_relevant_context(question):
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in docs])
    max_context_tokens = 256
    context_tokens = tokenizer.tokenize(context)
    if len(context_tokens) > max_context_tokens:
        context_tokens = context_tokens[:max_context_tokens]
        context = tokenizer.convert_tokens_to_string(context_tokens)
    sources = [doc.metadata for doc in docs]
    return context, sources

'''
Gera uma resposta para a pergunta utilizando o contexto recuperado
1. Chama get_relevant_context(question) para obter o contexto relevante e as fontes associadas à pergunta.
2. Cria um prompt estruturado que inclui a pergunta, o contexto recuperado e uma indicação para gerar a resposta:
3. Determina o número de tokens no prompt
4. Utiliza o pipeline nlp para gerar uma resposta baseada no prompt.
5. Remove o prompt original da resposta gerada para obter apenas a parte referente à resposta.
'''
def generate_response(question):
    context, sources = get_relevant_context(question)
    prompt = f"Question: {question}\nContext: {context}\nAnswer:"
    max_new_tokens = 512
    response = nlp(prompt, max_new_tokens=max_new_tokens, num_return_sequences=1)
    answer = response[0]['generated_text'][len(prompt):]
    return answer.strip(), sources


# Geração de Respostas
- Configure o modelo treinado para receber perguntas dos usuários.
Quando uma pergunta for recebida, utilize a integração RAG para recuperar informações relevantes do dataset AmazonTitles-1.3MM.

In [77]:
question = input("Please enter your question: ")
answer, sources = generate_response(question)
print("Answer:", answer)
print("\nSources:")
for source in sources:
    print(f"- UID: {source['uid']}, Title: {source['title']}")

Please enter your question: Tell me about sports
Answer: You can find a book or the article for that, read "Sports Illustrated is an Academic Sports Publishing Company and the Division 3 Basketball Association is an Academic Sports Publishing Company and the Division 3 Basketball Association is an Academic Sport Publishing Company and the Division 3 Basketball Association is an Academic Sport Publishing Company and the Division 3 Basketball Association is an Academic Sports Publishing Company and the Division 4 Basketball Association is an Academic Sports Publishing Company and the Division 3 Basketball Association is an Academic Sports Publishing Company and the Division 4 Basketball Association is an Academic Sport Publishing Company and the Division 4 Basketball Association is an Academic Sports Publishing Company and the Division 4 Basketball Association is an Academic Sports Publishing Company and the Division 4 Basketball Association is an Academic Sports Publishing Company and t